# Data Cleaning and Transformation of Web-Scraped Indeed Job Listings

### Data Description
A web-scraped data of job postings for Data Science roles from Indeed. Here's a breakdown of the columns:

* Title: A brief description or responsibility of the job.
* Title_URL: The URL of the job posting on Indeed's website.
* Title1: Job title.
* Company_name: The name of the company offering the job.
* Location: The location where the job is based.
* attribute_snippet: Salary information, formatted as a range.
* metadata__divcontains_class_attribute_snippet: Job Type.
* jobsnippet: Additional information about the job responsibilities or requirements.
* jobsnippet2: Additional information about the job responsibilities or requirements.
* date: The date when the job was posted. As the data was scraped in 14 Aug 2023, the date is based on 14 Aug 2023.
* Text: The full text of the job posting.

### Data Source
The data was scraped from Indeed's website using Octoparse 8.

### Import Necessary Libraries

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from IPython.display import display, HTML
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
import numpy as np
import json
import plotly.express as px


# Further refine the cleaning process to remove remaining postal codes and identifiers
import re
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 30)
# pd.set_option('display.width', 2)


In [3]:
df = pd.read_csv('DS_Job_ 2023 _ Indeed.csv')

In [4]:
df.head()

,Title,Title_URL,Title1,Company_name,Location,attribute_snippet,metadata__divcontains_class_attribute_snippet,jobsnippet,jobsnippet2,date,Text
0,Work directly with project scientists in ICPAC...,https://uk.indeed.com/rc/clk?jk=ed884653280c20...,"PDRA in weather prediction, seasonal climate p...",University of Oxford,Oxford,"£35,308 - £43,155 a year",NaN,scientists,Help set up an operational high-resolution for...,Posted\nPosted 17 days ago,\n \n Applications are invited for two Postdo...
1,Working to build effective data products that ...,https://uk.indeed.com/rc/clk?jk=4a70c1254d0492...,Junior Data Scientist,Penguin Books Limited,London SW1V,"£30,500 - £41,233 a year",NaN,data,NaN,Posted\nPosted 3 days ago,Job detailsHere’s how the job details align wi...
2,Share your knowledge and enthusiasm for data s...,https://uk.indeed.com/rc/clk?jk=f82754f93375de...,Data Scientist,Lidl,Hybrid remote in London KT5,"£56,100 - £78,100 a year",Full-time,data,The goal is to analyse business processes thro...,Posted\nPosted 2 days ago,Job detailsHere’s how the job details align wi...
3,Experience using consumer behavioural data pre...,https://uk.indeed.com/rc/clk?jk=8bd213b8a32179...,Data Scientist,Disney,London,NaN,NaN,data,Experience developing and scaling data science...,Posted\nPosted 4 days ago,\n \n Data Scientist \n \n Apply Later \n ...
4,The group has been working at the forefront of...,https://uk.indeed.com/rc/clk?jk=1e82e1285bc0d7...,Departmental Lecturer in Artificial Intelligen...,University of Oxford,Oxford OX3,"£45,585 - £54,395 a year",Full-time\n+1,NaN,NaN,Posted\nPosted 18 days ago,Job detailsHere’s how the job details align wi...


In [5]:
#In order to have better visualization of the dataframe, rearange and rename the columns
new_order = ['Company_name','Title1','Title_URL','Location','attribute_snippet','metadata__divcontains_class_attribute_snippet','Title','jobsnippet','jobsnippet2','date','Text']
df = df.reindex(columns=new_order)

# Rename columns
column_name_mapping = {
    'Title': 'Job_Summary',
    'Title_URL': 'Job_URL',
    'Title1': 'Job_Title',
    'Company_name': 'Company_Name',
    'Location': 'Job_Location',
    'attribute_snippet': 'Salary_Range',
    'metadata__divcontains_class_attribute_snippet': 'Job_Type',
    'jobsnippet': 'Primary_Skill',
    'jobsnippet2': 'Job_Description_Snippet',
    'date': 'Posting_Date',
    'Text': 'Full_Job_Description'
}

df = df.rename(columns=column_name_mapping)


In [6]:
# Display the first few rows with the new column names
df.head()

,Company_Name,Job_Title,Job_URL,Job_Location,Salary_Range,Job_Type,Job_Summary,Primary_Skill,Job_Description_Snippet,Posting_Date,Full_Job_Description
0,University of Oxford,"PDRA in weather prediction, seasonal climate p...",https://uk.indeed.com/rc/clk?jk=ed884653280c20...,Oxford,"£35,308 - £43,155 a year",NaN,Work directly with project scientists in ICPAC...,scientists,Help set up an operational high-resolution for...,Posted\nPosted 17 days ago,\n \n Applications are invited for two Postdo...
1,Penguin Books Limited,Junior Data Scientist,https://uk.indeed.com/rc/clk?jk=4a70c1254d0492...,London SW1V,"£30,500 - £41,233 a year",NaN,Working to build effective data products that ...,data,NaN,Posted\nPosted 3 days ago,Job detailsHere’s how the job details align wi...
2,Lidl,Data Scientist,https://uk.indeed.com/rc/clk?jk=f82754f93375de...,Hybrid remote in London KT5,"£56,100 - £78,100 a year",Full-time,Share your knowledge and enthusiasm for data s...,data,The goal is to analyse business processes thro...,Posted\nPosted 2 days ago,Job detailsHere’s how the job details align wi...
3,Disney,Data Scientist,https://uk.indeed.com/rc/clk?jk=8bd213b8a32179...,London,NaN,NaN,Experience using consumer behavioural data pre...,data,Experience developing and scaling data science...,Posted\nPosted 4 days ago,\n \n Data Scientist \n \n Apply Later \n ...
4,University of Oxford,Departmental Lecturer in Artificial Intelligen...,https://uk.indeed.com/rc/clk?jk=1e82e1285bc0d7...,Oxford OX3,"£45,585 - £54,395 a year",Full-time\n+1,The group has been working at the forefront of...,NaN,NaN,Posted\nPosted 18 days ago,Job detailsHere’s how the job details align wi...


In [7]:
# Check for duplicated rows based on the 'Job_URL' column
duplicated_rows_by_url = df[df.duplicated(subset='Job_URL', keep=False)]

duplicated_rows_by_url[['Job_URL']]

# Count the frequency of each URL in the 'Job_URL' column
url_value_counts = duplicated_rows_by_url['Job_URL'].value_counts()

url_value_counts


Series([], Name: Job_URL, dtype: int64)

In [8]:
# Check for any NA values in the dataframe
df.isna().sum()

Company_Name                 0
Job_Title                    0
Job_URL                      0
Job_Location                 0
Salary_Range                90
Job_Type                   116
Job_Summary                 17
Primary_Skill               97
Job_Description_Snippet    186
Posting_Date                 0
Full_Job_Description         1
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Company_Name             386 non-null    object
 1   Job_Title                386 non-null    object
 2   Job_URL                  386 non-null    object
 3   Job_Location             386 non-null    object
 4   Salary_Range             296 non-null    object
 5   Job_Type                 270 non-null    object
 6   Job_Summary              369 non-null    object
 7   Primary_Skill            289 non-null    object
 8   Job_Description_Snippet  200 non-null    object
 9   Posting_Date             386 non-null    object
 10  Full_Job_Description     385 non-null    object
dtypes: object(11)
memory usage: 33.3+ KB


### Job Title

In [10]:
df['Job_Title'].unique()

array(['PDRA in weather prediction, seasonal climate prediction, and artificial intelligence/machine learning x 2 posts',
       'Junior Data Scientist', 'Data Scientist',
       'Departmental Lecturer in Artificial Intelligence for Healthcare Imaging',
       'DATA SCIENTIST',
       'Research Assistant / Research Fellow in Health Data Science',
       'Assistant Professor AI, Innovation and Global Governance (EEI) - School of Government - 102147 - Grade 8',
       'Lead Health Data Scientist',
       'Associate Professor - Data Science (Research and Education) - School of Mathematics - 102259 - Grade 9',
       'Post Doctoral Research Associate in Data Science and Machine Learning',
       'Data Scientist, Marketing Effectiveness',
       'Graduate Data Scientist',
       'Data Scientist/Software Engineer (AI and Computer Vision)',
       'Assistant Professor - Data Science (Research and Education) - School of Mathematics - 102274 - Grade 8',
       'Lecturer/Senior Lecturer in Data 

In [11]:
df_job_title = df[['Job_Title']]

Extracts up to 20 keywords that are common across the titles using CountVectorizer.

This gives  a nice set of keywords that represent the core topics and themes in the job titles data. 

In [12]:
# Function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    text=text.replace("/"," ")
    # Remove non-alphabetic characters
    text = ''.join([char for char in text if char.isalpha() or char.isspace()])
    # Lemmatize and remove stopwords
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the job titles
df_job_title['Processed_Title'] = df_job_title['Job_Title'].apply(preprocess_text)

# Extract keywords using CountVectorizer
vectorizer = CountVectorizer(max_features=20)
X = vectorizer.fit_transform(df_job_title['Processed_Title'])
keywords = vectorizer.get_feature_names_out()

keywords


C:\Users\Justin\AppData\Local\Temp\ipykernel_5880\3104144893.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_title['Processed_Title'] = df_job_title['Job_Title'].apply(preprocess_text)


array(['ai', 'analyst', 'analytics', 'associate', 'data', 'engineer',
       'language', 'lead', 'learning', 'lecturer', 'machine', 'manager',
       'nlp', 'research', 'researcher', 'science', 'scientist', 'senior',
       'software', 'specialist'], dtype=object)

In [13]:
def categorize_job_titles(df, column_name):
    """
    Preprocess and categorize job titles based on domain and seniority.
    
    Parameters:
    - df: DataFrame containing the job titles.
    - column_name: Column name in the dataframe that contains the job titles.
    
    Returns:
    - DataFrame with additional columns for category, and refined seniority.
    """
    
    # Function to preprocess text
    def preprocess_text(text):
        # Convert to lowercase
        text = text.lower()
        text = text.replace("/", " ")
        # Remove non-alphabetic characters
        text = ''.join([char for char in text if char.isalpha() or char.isspace()])
        # Lemmatize and remove stopwords
        words = text.split()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
        return ' '.join(words)
    
    # Define a function to categorize job titles based on seniority
    def refined_categorize_seniority(title):
        if 'phd' in title:
            return 'PhD'
        elif any(term in title for term in["junior", "jr", "entry", "graduate", "trainee"]):
            return "Entry-level"
        elif any(term in title for term in ["senior researcher", "senior research"]):
            return "Senior Research"
        elif "senior lecturer" in title or "associate professor" in title:
            return "Senior Academic"
        elif any(term in title for term in ["senior", "sr", "snr"]):
            return "Senior"
        elif any(term in title for term in ["post-doc","postdoctoral","postdoctoral research","postdoctoral",'post doctoral']):
            return "Post-doc"
        elif any(term in title for term in ["fellow", "research fellow","research associate"]):
            return "Research Fellow/Associate"
        elif "research assistant" in title:
            return "Research Assistant"
        elif "lecturer" in title and not "senior lecturer" in title:
            return "Lecturer"
        elif any(term in title for term in ["lead", "head"]) and not any(term in title for term in ["manager", "director"]) and not any(term in title for term in ["consultancy", "leadership"]):
            return "Lead/Head"
        elif "manager" in title:
            return "Manager"
        elif "director" in title:
            return "Director"
        elif "intern" in title:
            return "Intern"
        elif "principal" in title:
            return "Principal"
        elif "associate" in title or "assistant" in title:
            return "Assistant/Associate"
        elif "research" in title:
            return "Researcher"
        elif "professor" in title:
            return "Lecturer"
        elif "tutor" in title:
            return "Tutor"
        else:
            return "Mid-Level"
        
    # Define a function to categorize job titles based on keywords
    # Adjust the domain categorization function to prioritize Machine Learning/AI when both categories are present
    def categorize_title(title):
        if any(term in title for term in ["machine", "learning", "ai","artificial intelligence","deep learning","artificial","ml"]):
            return "Machine Learning/AI"
        elif "nlp" in title or "natural language processing" in title:
            return "NLP"
        elif any(term in title for term in ["data", "analytics", "scientist", "science"]):
            return "Data Science"
        else:
            return "Other"
    
    # Apply preprocessing to the job titles
    df['Processed_Title'] = df[column_name].apply(preprocess_text)
    
    # Apply the domain categorization function to the preprocessed job titles
    df['Category'] = df['Processed_Title'].apply(categorize_title)
    
    # Apply the refined seniority categorization function to the preprocessed job titles
    df['Refined_Seniority'] = df['Processed_Title'].apply(refined_categorize_seniority)
    df.drop(columns=['Processed_Title'], inplace=True)
    return df



In [14]:
df = categorize_job_titles(df, 'Job_Title')

In [15]:
df.head()

,Company_Name,Job_Title,Job_URL,Job_Location,Salary_Range,Job_Type,Job_Summary,Primary_Skill,Job_Description_Snippet,Posting_Date,Full_Job_Description,Category,Refined_Seniority
0,University of Oxford,"PDRA in weather prediction, seasonal climate p...",https://uk.indeed.com/rc/clk?jk=ed884653280c20...,Oxford,"£35,308 - £43,155 a year",NaN,Work directly with project scientists in ICPAC...,scientists,Help set up an operational high-resolution for...,Posted\nPosted 17 days ago,\n \n Applications are invited for two Postdo...,Machine Learning/AI,Mid-Level
1,Penguin Books Limited,Junior Data Scientist,https://uk.indeed.com/rc/clk?jk=4a70c1254d0492...,London SW1V,"£30,500 - £41,233 a year",NaN,Working to build effective data products that ...,data,NaN,Posted\nPosted 3 days ago,Job detailsHere’s how the job details align wi...,Data Science,Entry-level
2,Lidl,Data Scientist,https://uk.indeed.com/rc/clk?jk=f82754f93375de...,Hybrid remote in London KT5,"£56,100 - £78,100 a year",Full-time,Share your knowledge and enthusiasm for data s...,data,The goal is to analyse business processes thro...,Posted\nPosted 2 days ago,Job detailsHere’s how the job details align wi...,Data Science,Mid-Level
3,Disney,Data Scientist,https://uk.indeed.com/rc/clk?jk=8bd213b8a32179...,London,NaN,NaN,Experience using consumer behavioural data pre...,data,Experience developing and scaling data science...,Posted\nPosted 4 days ago,\n \n Data Scientist \n \n Apply Later \n ...,Data Science,Mid-Level
4,University of Oxford,Departmental Lecturer in Artificial Intelligen...,https://uk.indeed.com/rc/clk?jk=1e82e1285bc0d7...,Oxford OX3,"£45,585 - £54,395 a year",Full-time\n+1,The group has been working at the forefront of...,NaN,NaN,Posted\nPosted 18 days ago,Job detailsHere’s how the job details align wi...,Machine Learning/AI,Lecturer


### Job Location

In [16]:
df['Job_Location'].unique()

array(['Oxford', 'London SW1V', 'Hybrid remote in London KT5', 'London',
       'Oxford OX3', 'Remote', 'Hybrid remote in Guildford', 'Exeter EX2',
       'Birmingham', 'Lincoln', 'Hybrid remote in London',
       'Hybrid remote in Didcot OX11', 'London EC1V', 'Hatfield',
       'Aberdeen', 'Cambridge', 'Hybrid remote in Manchester M13',
       'Hybrid remote in London E14', 'Hybrid remote in Bristol',
       'Edinburgh', 'Glasgow', 'Leicester LE1',
       'Hybrid remote in London NW9', 'England', 'Remote in London',
       'Sheffield', 'Remote in London N1', 'Remote in England',
       'Bristol BS8', 'Harpenden AL5',
       'Remote in Newcastle upon Tyne\n+1 location',
       'Hybrid remote in Leicestershire', 'Hybrid remote in Bristol BS37',
       'West Midlands', 'London W1D', 'Hybrid remote in Manchester',
       'Remote in Glasgow G2', 'Letchworth', 'Remote in London E1W',
       'Norwich', 'Wigan WN1', 'Bristol', 'Nottingham', 'Newbridge NP11',
       'Hybrid remote in Edinburgh

In [17]:
def clean_job_location(df):
    """
    Cleans the 'Job_Location' column of the dataframe and adds a 'Remote' column.
    
    Parameters:
    - df: DataFrame containing the job location details.
    
    Returns:
    - DataFrame with cleaned 'Job_Location' column and a new 'Remote' column.
    """
    
    # Create a new 'Remote' column based on the presence of the words "Remote" and "Hybrid"
    df['Remote'] = 'Not remote'
    df.loc[df['Job_Location'].str.contains('Remote', case=False), 'Remote'] = 'Remote'
    df.loc[df['Job_Location'].str.contains('Hybrid', case=False), 'Remote'] = 'Hybrid'
    
    # Clean up the 'Job_Location' column
    # df['Job_Location'] = df['Job_Location'].str.replace('Remote in ', '', case=False).str.replace('Hybrid remote in ', '', case=False)
    df['Job_Location'] = df['Job_Location'].str.replace('Remote in ', '').str.replace('Hybrid remote in ', '')
    

    def clean_location(location):
        # Remove any characters after city names like postal codes or alphanumeric characters
        location = re.split('([A-Z]{1,2}[0-9R][0-9A-Z]?)', location)[0]
        
        # Handle locations with multiple cities to keep only the primary city
        location = location.split('\n')[0]
        
        # Handle special cases
        if "University of" in location:
            location = location.replace("University of ", "")
        
      # Remove any unnecessary or redundant words
        for redundant_word in ["+1 location", "+2 locations", "+3 locations"]:
            location = location.replace(redundant_word, "")
        
        return location.strip()
    
    df["Job_Location"] = df["Job_Location"].apply(clean_location)
    
    return df



In [18]:
df = clean_job_location(df)

In [19]:
df['Job_Location'].unique()

array(['Oxford', 'London', 'Remote', 'Guildford', 'Exeter', 'Birmingham',
       'Lincoln', 'Didcot', 'Hatfield', 'Aberdeen', 'Cambridge',
       'Manchester', 'Bristol', 'Edinburgh', 'Glasgow', 'Leicester',
       'England', 'Sheffield', 'Harpenden', 'Newcastle upon Tyne',
       'Leicestershire', 'West Midlands', 'Letchworth', 'Norwich',
       'Wigan', 'Nottingham', 'Newbridge', 'Leeds', 'Dundee',
       'Hammersmith', 'Charlbury', 'Sidcup', 'High Wycombe',
       'Wolverhampton', 'Thame', 'Warrington', 'Reading', 'Plymouth',
       'Belfast', 'Warwick', 'Hullavington', 'East of England', 'Gaydon',
       'United Kingdom', 'Bollington', 'Southampton', 'Birkenhead',
       'West London', 'South West', 'Epsom', 'Bath', 'Stevenage',
       'Redhill', 'Oxfordshire', 'Cambridgeshire', 'Brentwood',
       'Guernsey', 'Lancashire', 'Cornwall', 'Osterley', 'Colchester',
       'Dunstable', 'Winnersh', 'Swindon', 'Haywards Heath', 'Wales',
       'Perth', 'St Albans', 'Headington', 'Coventry

### Salary_Range

In [20]:
df['Salary_Range'].unique()

array(['£35,308 - £43,155 a year', '£30,500 - £41,233 a year',
       '£56,100 - £78,100 a year', nan, '£45,585 - £54,395 a year',
       '£50,000 - £70,000 a year', '£30,487 - £37,099 a year',
       '£43,742 - £50,056 a year', '£51,306 - £59,450 a year',
       '£56,021 - £64,914 a year', '£37,099 a year', 'Full-time',
       '£20,595 - £23,121 a year', '£36,333 - £43,155 a year',
       'Fixed term contract', '£36,024 - £56,021 a year',
       '£60,000 - £75,000 a year', '£61,000 - £67,100 a year',
       '£45,585 - £57,696 a year', '£28,000 - £30,000 a year',
       '£65,000 - £95,000 a year', '£54,421 - £63,059 a year',
       '£44,414 - £52,841 a year', '£64,914 a year',
       '£44,263 - £54,395 a year', '£32,978 - £39,691 a year',
       '£80,000 a year', '£25,000 - £85,000 a year',
       '£46,974 - £56,021 a year', '£60,000 a year',
       '£45,791 - £50,855 a year', '£39,347 - £54,395 a year',
       '£50,000 - £90,000 a year', '£37,099 - £41,732 a year',
       '£37,386 - £

In [21]:
# Define a function to extract time units from the Salary_Range column
def extract_info_from_salary(salary):
    # Identify time unit
    time_unit = None
    if "year" in str(salary):
        time_unit = "Year"
    elif "month" in str(salary):
        time_unit = "Month"
    elif "week" in str(salary):
        time_unit = "Week"
    elif "day" in str(salary):
        time_unit = "Day"
    elif "hour" in str(salary):
        time_unit = "Hour"
    return time_unit
    
# Define the extract_salary function
def extract_salary(text):
    if not isinstance(text, str):
        return None  # or return an empty string, based on preference

    # Find the range with or without comma in numbers
    matches = re.findall(r'£\d{1,3}(?:,\d{3})*(?:\s*-\s*£\d{1,3}(?:,\d{3})*)?', text)
    return ' '.join(matches).strip()

def convert_to_int(salary):
    numbers = re.findall(r'\d+(?:[,_]\d+)*', salary)
    numbers = [int(num.replace(',', '')) for num in numbers]

    if '-' in salary:
        return numbers
    
    if not numbers:
        return None
        
    return numbers[0]

def fill_empty_and_na_values(df):
    """
    Fills missing and empty values in the 'Salary_Range' and 'Salary_Period' columns.
    
    Parameters:
    - df: DataFrame containing the job salary details.
    
    Returns:
    - DataFrame with filled values in 'Salary_Range' and 'Salary_Period' columns.
    """
    
    df['Salary_Range'].fillna("Not specified", inplace=True)
    df['Salary_Period'].fillna("Not specified", inplace=True)
    df['Salary_Range'].replace('', 'Not specified', inplace=True)
 
    return df

In [22]:
# Apply the function to the Salary_Range column
df['Salary_Period'] = df['Salary_Range'].map(extract_info_from_salary)
df['Salary_Range'] = df['Salary_Range'].apply(extract_salary)
# Fill the empty values with "Not specified"
df = fill_empty_and_na_values(df)

In [23]:
# Convert the salary range to a list of integers
df['Salary_Range'] = df['Salary_Range'].apply(convert_to_int)


In [24]:
# Check for empty values in the Salary_Range column
df.head()

,Company_Name,Job_Title,Job_URL,Job_Location,Salary_Range,Job_Type,Job_Summary,Primary_Skill,Job_Description_Snippet,Posting_Date,Full_Job_Description,Category,Refined_Seniority,Remote,Salary_Period
0,University of Oxford,"PDRA in weather prediction, seasonal climate p...",https://uk.indeed.com/rc/clk?jk=ed884653280c20...,Oxford,"[35308, 43155]",NaN,Work directly with project scientists in ICPAC...,scientists,Help set up an operational high-resolution for...,Posted\nPosted 17 days ago,\n \n Applications are invited for two Postdo...,Machine Learning/AI,Mid-Level,Not remote,Year
1,Penguin Books Limited,Junior Data Scientist,https://uk.indeed.com/rc/clk?jk=4a70c1254d0492...,London,"[30500, 41233]",NaN,Working to build effective data products that ...,data,NaN,Posted\nPosted 3 days ago,Job detailsHere’s how the job details align wi...,Data Science,Entry-level,Not remote,Year
2,Lidl,Data Scientist,https://uk.indeed.com/rc/clk?jk=f82754f93375de...,London,"[56100, 78100]",Full-time,Share your knowledge and enthusiasm for data s...,data,The goal is to analyse business processes thro...,Posted\nPosted 2 days ago,Job detailsHere’s how the job details align wi...,Data Science,Mid-Level,Hybrid,Year
3,Disney,Data Scientist,https://uk.indeed.com/rc/clk?jk=8bd213b8a32179...,London,None,NaN,Experience using consumer behavioural data pre...,data,Experience developing and scaling data science...,Posted\nPosted 4 days ago,\n \n Data Scientist \n \n Apply Later \n ...,Data Science,Mid-Level,Not remote,Not specified
4,University of Oxford,Departmental Lecturer in Artificial Intelligen...,https://uk.indeed.com/rc/clk?jk=1e82e1285bc0d7...,Oxford,"[45585, 54395]",Full-time\n+1,The group has been working at the forefront of...,NaN,NaN,Posted\nPosted 18 days ago,Job detailsHere’s how the job details align wi...,Machine Learning/AI,Lecturer,Not remote,Year


In [25]:
# Average salary
df['Average_Salary'] = df['Salary_Range'].apply(lambda x: np.mean(x) if x else np.nan)

### Job_Type

In [26]:
df['Job_Type'].unique()

array([nan, 'Full-time', 'Full-time\n+1', 'Permanent', 'Full-time\n+2',
       'Part-time', 'Fixed term contract', 'Temporary contract',
       'Internship', 'Freelance', 'Overtime', 'Apprenticeship',
       'Permanent\n+1'], dtype=object)

In [27]:
def clean_job_type(job_type):
    if isinstance(job_type, str):
        return re.sub(r'\n\+\d', '', job_type).strip()
    return job_type
def fill_empty_Job_Type(df):
    """
    Fills missing and empty values in the 'Salary_Range' and 'Salary_Period' columns.
    
    Parameters:
    - df: DataFrame containing the job salary details.
    
    Returns:
    - DataFrame with filled values in 'Salary_Range' and 'Salary_Period' columns.
    """
    
    df['Job_Type'].fillna("Not specified", inplace=True)  
    return df

In [28]:
# Apply the updated function
df['Job_Type'] = df['Job_Type'].apply(clean_job_type)
df=fill_empty_Job_Type(df)

In [29]:
df['Job_Type'].unique()

array(['Not specified', 'Full-time', 'Permanent', 'Part-time',
       'Fixed term contract', 'Temporary contract', 'Internship',
       'Freelance', 'Overtime', 'Apprenticeship'], dtype=object)

In [30]:
df.isna().sum()

Company_Name                 0
Job_Title                    0
Job_URL                      0
Job_Location                 0
Salary_Range               199
Job_Type                     0
Job_Summary                 17
Primary_Skill               97
Job_Description_Snippet    186
Posting_Date                 0
Full_Job_Description         1
Category                     0
Refined_Seniority            0
Remote                       0
Salary_Period                0
Average_Salary             199
dtype: int64

### Posting_Date

In [31]:
df['Posting_Date'].unique()

array(['Posted\nPosted 17 days ago', 'Posted\nPosted 3 days ago',
       'Posted\nPosted 2 days ago', 'Posted\nPosted 4 days ago',
       'Posted\nPosted 18 days ago', 'Posted\nPosted 30+ days ago',
       'Posted\nPosted 11 days ago', 'Posted\nPosted 19 days ago',
       'Posted\nPosted 16 days ago', 'Posted\nPosted 5 days ago',
       'Posted\nPosted 26 days ago', 'Posted\nPosted 6 days ago',
       'Posted\nPosted 7 days ago', 'Posted\nPosted 30 days ago',
       'Posted\nPosted 23 days ago', 'Employer\nActive 26 days ago',
       'Posted\nPosted 10 days ago', 'Posted\nPosted 25 days ago',
       'Posted\nPosted 9 days ago', 'Posted\nPosted 8 days ago',
       'Posted\nPosted 22 days ago', 'Posted\nPosted 24 days ago',
       'Posted\nPosted 20 days ago', 'Posted\nPosted 14 days ago',
       'Posted\nPosted 12 days ago', 'Posted\nPosted 1 day ago',
       'Posted\nPosted 28 days ago', 'Posted\nPosted 27 days ago',
       'Posted\nPosted 13 days ago', 'Employer\nActive 14 days ago',


In [32]:
# Cleaning Posting Date
def extract_days(text):
    # Extract number from the string
    result = re.search(r'(\d+)', text)
    if result:
        return int(result.group(1))
    return None

In [33]:
df['Posting_Date'] = df['Posting_Date'].apply(extract_days)

### Rearrange the column

In [34]:
new_order = ['Job_URL','Company_Name','Job_Title','Category','Refined_Seniority','Remote','Job_Location','Salary_Range','Salary_Period','Average_Salary','Job_Type','Posting_Date','Job_Summary','Primary_Skill','Job_Description_Snippet','Full_Job_Description']
# df_copy = df.copy()
# df_copy =df_copy.reindex(columns=new_order)
df = df.reindex(columns=new_order)

In [35]:
df.isna().sum()

Job_URL                      0
Company_Name                 0
Job_Title                    0
Category                     0
Refined_Seniority            0
Remote                       0
Job_Location                 0
Salary_Range               199
Salary_Period                0
Average_Salary             199
Job_Type                     0
Posting_Date                 0
Job_Summary                 17
Primary_Skill               97
Job_Description_Snippet    186
Full_Job_Description         1
dtype: int64

In [36]:
df['Primary_Skill'].unique()

array(['scientists', 'data', nan, 'Data', 'Scientists', 'Scientist'],
      dtype=object)

In [37]:
df['Job_Summary'].unique()

array(['Work directly with project scientists in ICPAC Kenya, and collaborate where needed with scientists at ECMWF.',
       "Working to build effective data products that drive real change in the business, you'll be involved at every step of the process, from exploratory data analysis…",
       'Share your knowledge and enthusiasm for data science with colleagues; help them to be more data-savvy.',
       'Experience using consumer behavioural data preferred.',
       'The group has been working at the forefront of international thinking in how to bring machine learning to ultrasound, with recent interests in multi-modal…',
       'Be recognised for your data wizardry and ability to tangibly talk about commercial discoveries.',
       'The Cancer Care group in the School of Health Sciences (University of Surrey) is excited to be advertising for a Research Assistant / Research Fellow to work on…',
       'To support development of rich data visualisation using industry tools.',
      

In [38]:
data = df.copy()

Skill Extract from the JOB DESCRIPTION

In [39]:
# List of data science-related skills and techniques
skills_list = [
    "Python", "R", "SQL", "Machine Learning", "Deep Learning", "Neural Networks", "TensorFlow", "Keras", "PyTorch",
    "NLP", "Big Data", "Hadoop", "Spark", "Tableau", "PowerBI", "Excel", "SAS", "MATLAB", "Java", "Scala",
    "Data Visualization", "Data Cleaning", "Data Engineering", "Data Analysis", "Statistics", "Time Series", "Clustering",
    "Regression", "Decision Trees", "Random Forest", "SVM", "API", "Cloud", "AWS", "Azure", "GCP", "Docker", "Kubernetes"
]

def regex_extract_skills(text):
    """Extract skills from the text using regular expressions."""
    """Refined regex-based skill extraction to accurately detect 'R'."""
    mentioned_skills = []
    
    for skill in skills_list:
        # Special handling for "R"
        if skill == "R":
            r_patterns = [
                r'\sR\s',  # Standalone 'R' not followed by a period and alphanumeric character
                r'Python/R',  # 'Python/R'
                r'R/Python',   # 'R/Python'
                r'R-Studio'    # 'R-Studio'
            ]
            if any(re.search(pattern, text, re.IGNORECASE) for pattern in r_patterns):
                mentioned_skills.append(skill)
        else:
            pattern = r'\b' + re.escape(skill.lower()) + r'\b'
            if re.search(pattern, text.lower()):
                mentioned_skills.append(skill)
    
    return ', '.join(mentioned_skills)


In [40]:
# Test the refined skill extraction on the Full_Job_Description column
# data['Regex_Skills_From_Full_Description'] = data['Full_Job_Description'].apply(lambda x: regex_extract_skills(str(x)))
df['Regex_Skills_From_Full_Description'] = df['Full_Job_Description'].apply(lambda x: regex_extract_skills(str(x)))

In [41]:
def count_skills_frequency(skill_column):
    """Count the frequency of each skill in the given pandas Series and return the sorted list."""
    # Initialize a Counter object
    skills_counter = Counter()
    
    # Update the counter for each set of skills
    for skill_set in skill_column:
        skills_counter.update(skill_set.split(', '))
    
    # Sort the skills by frequency
    sorted_skills = sorted(skills_counter.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_skills

In [42]:
count_skill = count_skills_frequency(df['Regex_Skills_From_Full_Description'])

In [43]:
count_skill

[('Python', 262),
 ('Machine Learning', 258),
 ('Statistics', 121),
 ('SQL', 120),
 ('Cloud', 94),
 ('Deep Learning', 76),
 ('Data Analysis', 68),
 ('R', 60),
 ('TensorFlow', 59),
 ('NLP', 56),
 ('PyTorch', 54),
 ('Big Data', 53),
 ('AWS', 51),
 ('Azure', 43),
 ('Java', 40),
 ('Spark', 34),
 ('Neural Networks', 29),
 ('Tableau', 28),
 ('Data Engineering', 28),
 ('Hadoop', 26),
 ('Regression', 25),
 ('', 24),
 ('Keras', 22),
 ('Time Series', 21),
 ('MATLAB', 21),
 ('GCP', 20),
 ('Docker', 18),
 ('Excel', 18),
 ('SAS', 16),
 ('Clustering', 16),
 ('Kubernetes', 15),
 ('Data Visualization', 13),
 ('API', 11),
 ('Scala', 9),
 ('PowerBI', 7),
 ('Decision Trees', 4),
 ('Data Cleaning', 4),
 ('Random Forest', 3),
 ('SVM', 2)]

To check if the 'R' capture correctly 

In [44]:
def extract_specific_r_context(text):
    """Extract a snippet of text around the specific detected 'R' patterns for context."""
    r_patterns = [
        r'.{0,20}\sR\s.{0,20}',  # Standalone 'R' not followed by a period and alphanumeric character
        r'.{0,20}Python/R.{0,20}',  # 'Python/R'
        r'.{0,20}R/Python.{0,20}',    # 'R or' (e.g., R or SAS)
        r'.{0,20}R-studio.{0,20}'
    ]
    
    # Find all matches for the R patterns
    matches = []
    for pattern in r_patterns:
        matches.extend(re.findall(pattern, text, re.IGNORECASE))
    
    # Join multiple matches (if any) with ' ... ' for clarity
    return ' ... '.join(matches)

In [46]:
data = df.copy()

In [ ]:
# Extract the specific context in which "R" appears from the Full_Job_Description column based on the refined regex extraction method
r_context_entries_refined = data[data['Regex_Skills_From_Full_Description'].str.contains('R', na=False, case=False)]
r_context_from_refined_extraction = r_context_entries_refined['Full_Job_Description'].apply(lambda x: extract_specific_r_context(str(x)))

In [51]:
# Display the extracted specific context based on the refined regex extraction method
data['R_context'] = r_context_from_refined_extraction
data[data['Regex_Skills_From_Full_Description'].str.contains(r'\bR\b', na=False, case=True, regex=True)].head()

,Job_URL,Company_Name,Job_Title,Category,Refined_Seniority,Remote,Job_Location,Salary_Range,Salary_Period,Average_Salary,Job_Type,Posting_Date,Job_Summary,Primary_Skill,Job_Description_Snippet,Full_Job_Description,Regex_Skills_From_Full_Description,R_context
3,https://uk.indeed.com/rc/clk?jk=8bd213b8a32179...,Disney,Data Scientist,Data Science,Mid-Level,Not remote,London,None,Not specified,NaN,Not specified,4,Experience using consumer behavioural data pre...,data,Experience developing and scaling data science...,\n \n Data Scientist \n \n Apply Later \n ...,"Python, R, SQL, Statistics","xperience in Python, R or another statistic"
7,https://uk.indeed.com/rc/clk?jk=671c27caa3bb60...,Devon Partnership NHS Trust,Data Scientist,Data Science,Mid-Level,Not remote,Exeter,"[43742, 50056]",Year,46899.0,Permanent,2,To support development of rich data visualisat...,data,The post holder will pro-actively identify opp...,Job details\nHere’s how the job details align ...,"Python, R",g techniques such as R & Python. The post h
12,https://uk.indeed.com/rc/clk?jk=72d8aa70c30b88...,LEGO,"Data Scientist, Marketing Effectiveness",Data Science,Mid-Level,Hybrid,London,None,Not specified,NaN,Not specified,30,"Knowledge of data visualisation tools, Tableau...",data,Cleaning and interrogating large data sets to ...,\n \n \n \n Location\n \n \n Lon...,"Python, R, SQL, Big Data, Hadoop, Tableau, Sta...","/HQL (Hadoop, Hive), R or Python. It will b"
17,https://uk.indeed.com/rc/clk?jk=8960cabd3742fd...,JATO,Data Scientist,Data Science,Mid-Level,Remote,Remote,None,Not specified,NaN,Full-time,18,Independent proficiency in the data science pr...,data,NaN,Job detailsHere’s how the job details align wi...,"Python, R, Machine Learning, NLP, Data Enginee...",Proficiency using Python/R
19,https://uk.indeed.com/rc/clk?jk=89807539723295...,ERM Group,Graduate Data Scientist (Entry Level),Data Science,Entry-level,Not remote,Aberdeen,None,Not specified,NaN,Full-time,6,Support the undertaking of complex data analyt...,data,Have a proven ability in big data analysis and...,Job detailsHere’s how the job details align wi...,"Python, R, Machine Learning, Big Data, Data An...",ent in Python and/or R (other programming l


### Plot the job opening by location

In [52]:
df['Job_Location'].unique()

array(['Oxford', 'London', 'Remote', 'Guildford', 'Exeter', 'Birmingham',
       'Lincoln', 'Didcot', 'Hatfield', 'Aberdeen', 'Cambridge',
       'Manchester', 'Bristol', 'Edinburgh', 'Glasgow', 'Leicester',
       'England', 'Sheffield', 'Harpenden', 'Newcastle upon Tyne',
       'Leicestershire', 'West Midlands', 'Letchworth', 'Norwich',
       'Wigan', 'Nottingham', 'Newbridge', 'Leeds', 'Dundee',
       'Hammersmith', 'Charlbury', 'Sidcup', 'High Wycombe',
       'Wolverhampton', 'Thame', 'Warrington', 'Reading', 'Plymouth',
       'Belfast', 'Warwick', 'Hullavington', 'East of England', 'Gaydon',
       'United Kingdom', 'Bollington', 'Southampton', 'Birkenhead',
       'West London', 'South West', 'Epsom', 'Bath', 'Stevenage',
       'Redhill', 'Oxfordshire', 'Cambridgeshire', 'Brentwood',
       'Guernsey', 'Lancashire', 'Cornwall', 'Osterley', 'Colchester',
       'Dunstable', 'Winnersh', 'Swindon', 'Haywards Heath', 'Wales',
       'Perth', 'St Albans', 'Headington', 'Coventry

In [53]:
locations = df['Job_Location'].value_counts()
locations_df = pd.DataFrame(locations)
locations_df.index.name = 'city'
locations_df.head()

,Job_Location
city,
London,169
Cambridge,15
Remote,14
Manchester,14
Bristol,11


In [54]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="myapp")

coords = {loc: geolocator.geocode(loc) for loc in locations.index}
coords = {loc: (c.latitude, c.longitude) for loc, c in coords.items() if c}

In [48]:
coords

{'London': (51.5073359, -0.12765),
 'Cambridge': (52.2055314, 0.1186637),
 'Remote': (43.0059455, -123.8925908),
 'Manchester': (53.4794892, -2.2451148),
 'Bristol': (51.4538022, -2.5972985),
 'Oxford': (51.7520131, -1.2578499),
 'Birmingham': (52.4796992, -1.9026911),
 'Leeds': (53.7974185, -1.5437941),
 'Edinburgh': (55.9533456, -3.1883749),
 'Reading': (51.4564242, -0.9700664),
 'England': (52.5310214, -1.2649062),
 'Glasgow': (55.861155, -4.2501687),
 'Harpenden': (51.8145103, -0.3549311),
 'Sheffield': (53.3806626, -1.4702278),
 'Leicester': (52.6362, -1.1331969),
 'Nottingham': (52.9534193, -1.1496461),
 'Aberdeen': (57.1482429, -2.0928095),
 'Guildford': (51.2356068, -0.5732063),
 'West London': (38.1943567, -81.3686944),
 'Lincoln': (40.8088861, -96.7077751),
 'Belfast': (54.596391, -5.9301829),
 'United Kingdom': (54.7023545, -3.2765753),
 'High Wycombe': (51.627422, -0.7484153),
 'South West': (5.3356587, 100.2389496),
 'Warwick': (52.2814519, -1.5815742),
 'Newcastle upon Ty

In [55]:
# Convert the dictionary to a DataFrame
lat_lng_df = pd.DataFrame.from_dict(coords, orient='index', columns=['Latitude', 'Longitude'])
# Rename the index to 'City'
lat_lng_df.index.name = 'city'
lat_lng_df.rename({'Latitude': 'lat', 'Longitude': 'lng'}, axis='columns', inplace=True)

In [56]:
lat_lng_df.head()

,lat,lng
city,,
London,51.507336,-0.127650
Cambridge,52.205531,0.118664
Remote,43.005946,-123.892591
Manchester,53.479489,-2.245115
Bristol,51.453802,-2.597298


In [58]:
open_job = pd.merge(locations_df, lat_lng_df, on="city")
open_job.reset_index(inplace=True)
open_job.head()


,city,Job_Location,lat,lng
0,London,169,51.507336,-0.127650
1,Cambridge,15,52.205531,0.118664
2,Remote,14,43.005946,-123.892591
3,Manchester,14,53.479489,-2.245115
4,Bristol,11,51.453802,-2.597298


In [59]:
# Plotting the Geographical Distribution of Job Openings in the UK
top_10_cities = open_job.nlargest(10, 'Job_Location')
top_10_cities


,city,Job_Location,lat,lng
0,London,169,51.507336,-0.127650
1,Cambridge,15,52.205531,0.118664
2,Remote,14,43.005946,-123.892591
3,Manchester,14,53.479489,-2.245115
4,Bristol,11,51.453802,-2.597298
5,Oxford,10,51.752013,-1.257850
6,Birmingham,10,52.479699,-1.902691
7,Leeds,9,53.797418,-1.543794
8,Edinburgh,9,55.953346,-3.188375
9,Reading,8,51.456424,-0.970066


In [60]:
import folium

# Create a map centered around coordinates
m = folium.Map(location=[53.483959, -1.244644], zoom_start=6)

# Add points to the map
for i, row in open_job.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=row['Job_Location'] / 10,  # Adjust the scaling factor as needed
        color='blue',
        fill=True,
        fill_color='blue',
        tooltip=row['city']  # Tooltip to show on hover
    ).add_to(m)

# Annotate top 10 cities
top_10_cities = open_job.nlargest(10, 'Job_Location')
for i, row in top_10_cities.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        tooltip=row['city']
    ).add_to(m)

m.save('map_with_tooltip.html')



In [61]:
m